In [ ]:
import networkx as nx
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
%matplotlib notebook
import csv
import os
import itertools

In [ ]:
from scripts.utility import *
from scripts.features import *

# Features
It is a good idea to save our features in text files, so that we only have to calculate them once. Later on when we employ clustering, we can just read the values off of these files instead of iterating over the graphs again, which is time consuming.

## 1. Geometrical

### 1.1 Number of nodes and edges

In [ ]:
# Calculate the number of nodes and edges for each network and save them to files
with open('features/n_nodes.txt', 'w') as node_file, open('features/n_edges.txt', 'w') as edge_file:
    for network_id, G in graph_generator():
        node_file.write(network_id + '\t' + str(nx.number_of_nodes(G)) + '\n') # nx.number_of_nodes() function
        edge_file.write(network_id + '\t' + str(nx.number_of_edges(G)) + '\n') # use tab character \t, not two spaces
        
###NOTE: Have to pull features from github before running script
##IDEA: Can calculate every feature on the same loop

### 1.2 Average node degree

In [ ]:
# Calculate the average node degree, i.e. the average number of edges that are connected to a node,
# for each network and write them to a file
with open('features/average_node_degree.txt', 'w') as file:
    for network_id, G in graph_generator(): # check this access to graph_generator
        average_node_degree = np.mean(list(G.degree().values()))
        file.write(network_id + '\t' + str(average_node_degree) + '\n')

### 1.3 Vein density

Vein density **$\sigma$** is calculated by summing the total length of all veins and dividing by total leaf area.


In [ ]:
save_feature(vein_density, skip_existing=True, clean=False)

### 1.4 Areole area

Areole area **$A$** is obtained calculating basic cycle areas using the standard formula for the area of a general polygon.

In [ ]:
save_feature(areole_area, skip_existing=True, clean=False)

### 1.5 Areole density 

Areole density **$\rho_A$** is the total number of areoles divided by leaf area.

In [ ]:
save_feature(areole_density, skip_existing=True, clean=False)

### 1.6 Weighted vein thickness

Weighted vein thickness **$d$** is calculated as the total sum of the product radius*length of each
    individual vein segment divided by total vein length

In [ ]:
save_feature(weighted_vein_thickness, skip_existing=True, clean=False)

## 2. Topological

### 2.1 Nesting number

In [ ]:
save_feature(nesting_numbers, skip_existing=True, clean=True)

# Data Analysis

In [ ]:
generator = graph_generator()

In [ ]:
_, G = next(generator)

In [ ]:
coords = np.sqrt(2) * np.array([[0,0.5], [0.5,1], [0.5,0], [1,0.5]], dtype='float64')
#coords = np.array([[0,0], [2,0], [0,1], [2,1]], dtype='float64')
vein_distance(G)

## Principal Component Analysis

In [ ]:
data = read_features()

In [ ]:
feature_names = [
    'topological_length',  
    #'nesting_number_weighted_no_ext',
    'nesting_number_unweighted',
    'nesting_number_weighted',
    #'nesting_number_unweighted_no_ext',
    'vein_density',
    'vein_distance',
    'areole_area',
    'areole_density',
    'weighted_vein_thickness',
    #'average_node_degree',
    #'n_edges',
    #'n_nodes',
    
]

feature_labels = {
    'topological_length'              : r'$L_{top}$',
    'nesting_number_weighted'         : r'$i_w$',
    'nesting_number_weighted_no_ext'  : r'$i_w^{no}$',
    'nesting_number_unweighted'       : r'$i_u$',
    'nesting_number_unweighted_no_ext': r'$i_u^{no}$',
    'vein_density'                    : r'$\sigma$',
    'vein_distance'                   : r'$a$',
    'areole_area'                     : r'$A$',
    'areole_density'                  : r'$\rho_A$',
    'weighted_vein_thickness'         : r'$d$',
    'average_node_degree'             : r'$deg_N$',
    'n_edges'                         : r'$N_e$',
    'n_nodes'                         : r'$N_n$',
}

In [ ]:
network_ids = sorted(data['nesting_number_weighted'].keys())
X = []
for feature_name in feature_names:
    row = [data[feature_name][network_id] for network_id in network_ids]
    X.append(row)
X = np.array(X)
X = X.T
X = X - np.mean(X, axis=0)
X1 = X / np.std(X, axis=0)
X2 = X / np.max(X, axis=0)

np.std(X2, axis=0) / np.sum(np.std(X2, axis=0))

In [ ]:
for X in [X1, X2]:
    pca = PCA()
    pca.fit(X)
    
    normalized_eigenvalues = pca.explained_variance_ratio_
    print(normalized_eigenvalues)
    
    principal_components = pca.components_
    print(principal_components)
    
    scaled_principal_components = (normalized_eigenvalues * principal_components.T).T
    #print(scaled_principal_components)
    
    relative_feature_contributions = np.sum(abs(scaled_principal_components), axis=0)
    #print(relative_feature_contributions)
    
    
    fig, ax = plt.subplots()
    x = np.arange(len(relative_feature_contributions))
    ax.bar(x, relative_feature_contributions)
    labels = [feature_labels[name] for name in feature_names]
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    
    
    width = 0.35

    fig, ax = plt.subplots()
    x = np.arange(len(relative_feature_contributions))
    ax.bar(x, abs(principal_components[0,:]), width)
    ax.bar(x + width, abs(principal_components[1,:]), width)
    #ax.bar(x + 2*width, abs(principal_components[2,:]), width)

    labels = [feature_labels[name] for name in feature_names]
    ax.set_xticks(x + width/2)
    ax.set_xticklabels(labels)

In [ ]:
save_feature(vein_distance, skip_existing=True, clean=True)